# Test access to vision_server and rq_xmlrpcserver

In [ ]:
import requests
from typing import Tuple

def get_vs(uri) -> Tuple[str, dict]:
  response = requests.get("http://10.30.5.158:4242/" + uri)
  response.raise_for_status()
  return response.json()

In [ ]:
# XMLRPC client to connect and list methods from 10.30.5.158:4242
import requests
import xmlrpc.client
import json

# rq_xmlrpcserver
print("rq_xmlrpcserver")
with xmlrpc.client.ServerProxy("http://10.30.5.158:4250/") as proxy:
  try:
    print(proxy.getserverversion())
  except xmlrpc.client.Fault as err:
    print("A fault occurred")
    print("Fault code: %d" % err.faultCode)
    print("Fault string: %s" % err.faultString)
    
# vision_server
print("\n\nvision_server")
for cmd in [
    'getcamerasettings', 
    # 'setcamerasettings', 'setdefaultcamerasettings',
    # 'status', 'getbarcodelicense', 'hello',
    # 'create', 'resetzoomregion', 'setdisplaysize', 'process',
    # 'trigger', 'triggerinprogress', 'resetserverstate',
    # 'beginbarcode', 'rejectmodule', 'endoperationmode', 'streamoutmodule',
    # 'beginoperationmode', 'endoperationmode', 'streaminmodule', 
    
  ]:
  try:
    print(f'{cmd}:\n{json.dumps(get_vs(cmd), indent=2, sort_keys=True)}')
  except requests.exceptions.HTTPError as err:
    print("HTTP error occurred: %s" % err)
  except Exception as err:
    print("Other error occurred: %s" % err)


# Check changes and keys in camera settings

In [ ]:
valbef = None

In [ ]:
import copy

vals = get_vs('getcamerasettings')

# Print all values different from valbef
print('Diff with last values:')
if valbef is None:
  print(vals)
else:
  for key in vals['cameraSettings']:
    if vals['cameraSettings'][key] != valbef['cameraSettings'][key]:
      print(f'{key}: {vals["cameraSettings"][key]}')
      
# Print all keys contianing 'focus
kw = 'focus'
print(f'\nKeys containing {kw}:')
for key in vals['cameraSettings']:
  if kw in key:
    print(f'{key}: {vals["cameraSettings"][key]}')

valbef = copy.copy(vals)

# Setup and test settings

In [ ]:
# Config Jupyter to force reload modules (DEV only)
%load_ext autoreload
%autoreload 2

In [ ]:
from URBasic import ISCoin, CameraSettings
iscoin = ISCoin(host="10.30.5.158", opened_gripper_size_mm=40)
iscoin.camera.resetCameraSettings()

In [ ]:
from URBasic import RobotiqWristCamera
iscoin.displayCameraStreamOCVParallel() # RobotiqWristCamera.ImageType.MAGNITUDE)

In [ ]:
# Plays with camera LEDs
from time import sleep

cs = CameraSettings()
while(True):
  for i in range(0,11):
    print(f'Setting manual mode to {10*i}')
    cs.lightingSettings.setManualMode(10*i)
    # Actually send changes
    if not iscoin.camera.setCameraSettings(cs):
      print('Error setting camera settings')
      break
    sleep(0.2)
  print('Setting off mode')
  cs.lightingSettings.setOffMode()
  iscoin.camera.setCameraSettings(cs)
  sleep(3)

In [ ]:
# Greenish tone
cs = CameraSettings()
cs.whiteBalanceSettings.setManualMode(2000, 4000, 2000)
iscoin.camera.setCameraSettings(cs)

In [ ]:
# Reset
cs = CameraSettings()
iscoin.camera.resetCameraSettings()